In [ ]:
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install einops
!pip install faiss-gpu
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub

!pip install --upgrade --quiet  wikipedia
!pip install --upgrade --quiet  arxiv
!pip install --upgrade --quiet  pymupdf

!pip install xmltodict

!pip install unstructured
!pip install PyMuPDF
!pip install pdfminer.six
!pip install qdrant-client
!pip install sentence_transformers
!pip install pi_heif
!pip install unstructured_inference

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging
import os
logging.basicConfig(level=logging.INFO)
from google.colab import userdata
import locale

import torch
import os
import bs4
import json
import numpy as np
import time
import pandas as pd

from pprint import pprint

import locale
import uuid


import fitz
import requests
from langchain.embeddings import HuggingFaceEmbeddings
from typing import Tuple, List, Optional
from pathlib import Path
import tempfile
from google.colab import files
import requests
from typing import Optional, Dict
from collections import defaultdict
from typing import List, Dict
import subprocess
import pkg_resources


from transformers import AutoTokenizer , AutoModelForCausalLM
from transformers import pipeline, BitsAndBytesConfig

from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import Qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.utils.math import cosine_similarity
from langchain.schema import Document

from langchain_community.document_loaders import ArxivLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PubMedLoader
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.chat_models import ChatCohere

from langchain.document_loaders import PyMuPDFLoader
from langchain.prompts import ChatPromptTemplate

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
from google.colab import userdata
from google.colab import drive
import re


<ipython-input-1-3cc5fbafaee5>:33: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [ ]:
drive.mount('/content/drive')
locale.getpreferredencoding = lambda: "UTF-8"

Mounted at /content/drive


In [ ]:
# Define paths dynamically based on the company name
# policy_paths = f"/content/drive/MyDrive/210 Capstone/policy/company_name}_eu.pdf"
# #gdpr_url = "https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32016R0679"
# gdpr_drive_path = "/content/drive/MyDrive/210 Capstone/GDPR.pdf"
base_embeddings = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

QDRANT_API_KEY = userdata.get('QDRANT_API_KEY')



In [ ]:
def fn_local_with_paths(input_data: Dict) -> Tuple[Optional[str], Optional[str], str, str, str]:
    """
    Retrieve the uploaded policy file and GDPR document from Google Drive.

    Args:
        input_data (dict): Input JSON containing `file_name`, `company_name`, `uuid`, and `question_type`.

    Returns:
        tuple: Paths to policy document, GDPR document, company name, uuid, and document type.
    """
    print("Received input data: ", input_data)

    # Extract question_type and uuid
    question_type = input_data.get("question_type", "Policy")
    uuid = input_data.get("uuid")
    if not uuid:
        raise ValueError("Input JSON payload missing 'uuid' field")

    gdpr_drive_path = "/content/drive/MyDrive/210 Capstone/policy/GDPR.pdf"

    if question_type == "Policy":
        company_name = input_data.get("company_name", "").strip()
        if not company_name:
            raise ValueError("Input JSON payload missing 'company_name' field for Policy questions.")
        policy_paths = f"/content/drive/MyDrive/210 Capstone/policy/{company_name}.pdf"
        if not os.path.exists(policy_paths):
            raise FileNotFoundError(f"The policy file for {company_name} was not found at {policy_paths}")
        print(f"Policy file located at: {policy_paths}")
    else:
        company_name = None
        policy_paths = None

    if question_type == "GDPR" and not os.path.exists(gdpr_drive_path):
        raise FileNotFoundError(f"The GDPR document was not found at {gdpr_drive_path}")

    print(f"GDPR document located at: {gdpr_drive_path}")

    return policy_paths, gdpr_drive_path, company_name, uuid, question_type

In [ ]:
# Define the function to load PDF content
def load_pdf_content_with_page_count(pdf_path: str) -> Tuple[List[str], int]:
    """
    Load the content of a PDF and return the text from each page along with the page count.

    Args:
        pdf_path: Path to the PDF file.

    Returns:
        tuple: (List of page contents as strings, Total number of pages)
    """
    try:
        document = fitz.open(pdf_path)
        #print(f"Document opened successfully. Page count: {len(document)}")
        contents = [page.get_text() for page in document]
        num_pages = len(contents)
        document.close()
        return contents, num_pages
    except Exception as e:
        logger.error(f"Failed to load PDF content from {pdf_path}: {e}")
        return [], 0


# Define the function to load policy and GDPR documents
def load_policy_document(policy_path: str, gdpr_path: str) -> Tuple[List[str], List[str]]:
    """
    Load a single policy document and the GDPR document with proper error handling.

    Parameters:
    - policy_path (str): Path to the company policy PDF.
    - gdpr_path (str): Path to the GDPR reference PDF.

    Returns:
    - tuple: (list of policy content as strings, gdpr contents)
    """
    base_embeddings = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")
    if base_embeddings is None:
        logger.error("Failed to initialize embeddings")
        return [], []

    policy_contents, policy_pages = load_pdf_content_with_page_count(policy_path)
    logger.info(f"Loaded policy document ({Path(policy_path).name}): {policy_pages} pages")

    # Load GDPR document
    gdpr_contents, gdpr_pages = load_pdf_content_with_page_count(gdpr_path)
    logger.info(f"Loaded GDPR document ({Path(gdpr_path).name}): {gdpr_pages} pages")

    return policy_contents, gdpr_contents

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def extract_formatted_text(pdf_path: str) -> List[Dict]:
    """
    Extract text with its formatting information from PDF.
    Returns list of dictionaries containing text and its formatting properties.
    """
    formatted_blocks = []
    try:
        doc = fitz.open(pdf_path)
        position = 0

        for page_num, page in enumerate(doc):
            blocks = page.get_text("dict")["blocks"]
            prev_y1 = None

            for block in blocks:
                if "lines" in block:
                    for line in block["lines"]:
                        y0 = line["bbox"][1]
                        line_spacing = y0 - prev_y1 if prev_y1 is not None else 0
                        prev_y1 = line["bbox"][3]

                        for span in line["spans"]:
                            text = span["text"].strip()
                            if text:
                                formatted_blocks.append({
                                    "text": text,
                                    "font_name": span["font"],
                                    "font_size": span["size"],
                                    "is_bold": "bold" in span["font"].lower() or span["flags"] & 2**4 != 0,
                                    "line_spacing": line_spacing,
                                    "position": position,
                                    "page_num": page_num + 1
                                })
                            position += len(text) + 1

        return formatted_blocks
    except Exception as e:
        print(f"Error extracting formatted text: {e}")
        return []

def analyze_document_formatting(blocks: List[Dict]) -> Dict:
    """
    Analyze document formatting to establish baseline metrics.
    """
    stats = defaultdict(list)

    for block in blocks:
        stats["font_sizes"].append(block["font_size"])
        stats["line_spacings"].append(block["line_spacing"])

    return {
        "avg_font_size": sum(stats["font_sizes"]) / len(stats["font_sizes"]),
        "max_font_size": max(stats["font_sizes"]),
        "avg_line_spacing": sum(stats["line_spacings"]) / len(stats["line_spacings"]) if stats["line_spacings"] else 0
    }

def identify_potential_headers(blocks: List[Dict], format_stats: Dict) -> List[Dict]:
    potential_headers = []
    print("calculating potential headers")

    for block in blocks:
        formatting_score = 0
        characteristics = []

        # Format-based scoring
        if block["font_size"] > format_stats["avg_font_size"]:
            formatting_score += 2
            characteristics.append("larger_font")

        if block["is_bold"]:
            formatting_score += 2
            characteristics.append("bold")

        if block["line_spacing"] > format_stats["avg_line_spacing"] * 1.5:
            formatting_score += 1
            characteristics.append("increased_spacing")

        word_count = len(block["text"].split())
        if word_count <= 10:
            formatting_score += 1
            characteristics.append("short_text")

        if block["text"].istitle() or block["text"].isupper():
            formatting_score += 1
            characteristics.append("title_case_or_caps")

        if formatting_score >= 3:
            potential_headers.append({
                "header": block["text"],
                "position": block["position"]
            })

    return potential_headers

def chunk_policy_by_headers(headers: List[Dict], blocks: List[Dict]) -> List[Dict]:
    """
    Chunk the policy text based on section headers using the original formatted blocks.
    """
    chunks = []
    full_text = ""
    current_position = 0

    for block in blocks:
        full_text += block["text"] + "\n"

    for i, header_data in enumerate(headers):
        start_pos = header_data["position"]
        end_pos = headers[i + 1]["position"] if i + 1 < len(headers) else len(full_text)
        chunk_text = full_text[start_pos:end_pos].strip()

        chunks.append({
            "header": header_data["header"],
            "content": chunk_text
        })

    return chunks

def process_policy(pdf_path: str) -> Dict:
    """
    Process a single PDF policy document and extract sections based on formatting only.
    No GPT validation needed.
    """
    try:
        # Extract text with formatting
        formatted_blocks = extract_formatted_text(pdf_path)
        if not formatted_blocks:
            return {"status": "error", "pdf_path": pdf_path, "message": "Failed to extract formatted text"}

        # Analyze document formatting
        format_stats = analyze_document_formatting(formatted_blocks)

        # Get headers based on formatting only
        confirmed_headers = identify_potential_headers(formatted_blocks, format_stats)

        if not confirmed_headers:
            return {"status": "error", "pdf_path": pdf_path, "message": "No headers found in the document"}

        chunks = chunk_policy_by_headers(confirmed_headers, formatted_blocks)

        return {
            "status": "success",
            "pdf_path": pdf_path,
            "total_chunks": len(chunks),
            "chunks": chunks
        }

    except Exception as e:
        return {"status": "error", "pdf_path": pdf_path, "message": str(e)}


def further_chunk_policy(company_name: str, uuid: str, policy: Dict, chunk_size: int = 1000, chunk_overlap: int = 50) -> List[Dict]:
    """
    Further split each chunk's content for a single company policy using a Recursive Text Splitter.

    Args:
        policy (Dict): A dictionary representing the policy document with its chunks.
        chunk_size (int): The maximum size of each sub-chunk.
        chunk_overlap (int): The overlap between consecutive sub-chunks.

    Returns:
        List[Dict]: A list of refined sub-chunks with metadata.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False
    )
    refined_chunks = []

    #pdf_path = policy["pdf_path"]
    #company_name = policy_paths.split('/')[-1].replace('.pdf', '').capitalize()

    for chunk in policy["chunks"]:
        try:
            sub_chunks = text_splitter.split_text(chunk["content"])

            for i, sub_chunk in enumerate(sub_chunks, 1):
                refined_chunks.append({
                    "page_content": sub_chunk,
                    "metadata": {
                        "company_name": company_name,
                        "uuid": uuid,
                        "pdf_path": policy["pdf_path"],
                        "header": chunk["header"],
                        "chunk_index": i,
                        "total_sub_chunks": len(sub_chunks),
                        "document_type": "Policy"
                    }
                })
        except Exception as e:
            print(f"Error processing chunk for company {company_name} with header '{chunk['header']}': {str(e)}")
            continue

    return refined_chunks


def chunk_gdpr_by_section(gdpr_contents: List[str]) -> List[Document]:
    """
    Chunk GDPR text into sections based on articles and recitals, without mapping to policy categories.

    Parameters:
    - gdpr_contents (List[str]): List containing GDPR text.

    Returns:
    - List[Document]: List of Document objects, each representing a chunk of a GDPR section.
    """
    try:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)
        documents = []
        full_text = "\n".join(gdpr_contents)

        recital_pattern = re.compile(r"\((\d+)\)\s", re.MULTILINE)
        article_pattern = re.compile(r"(Article\s(\d+))\b", re.IGNORECASE)

        all_matches = [(match.start(), match.group(1), 'recital') for match in re.finditer(recital_pattern, full_text)]
        all_matches += [(match.start(), match.group(1), 'article', match.group(2)) for match in re.finditer(article_pattern, full_text)]
        all_matches.sort()

        for i, (start, header, section_type, *article_number) in enumerate(all_matches):
            end = all_matches[i + 1][0] if i + 1 < len(all_matches) else len(full_text)
            section_text = full_text[start:end].strip()

            section_header = None

            if section_type == 'article' and article_number:
                article_number = article_number[0].strip()
                section_header = f"Article {article_number}"

            elif section_type == 'recital':
                section_header = f"Recital {header}"

            chunks = text_splitter.split_text(section_text)

            for chunk in chunks:
                documents.append(Document(
                    page_content=chunk,
                    metadata={
                        "header": section_header
                    }
                ))

        print(f"\nTotal documents created: {len(documents)}")
        return documents

    except Exception as e:
        print(f"Error processing GDPR contents: {e}")
        return []


In [ ]:
def initialize_falcon_pipeline(
    model_name: str = "tiiuae/falcon-7b-instruct",
    max_new_tokens: int = 1500,
    temperature: float = 0.55,
    top_p: float = 0.9,
    repetition_penalty: float = 1.3,
    offload_folder: str = "offload_folder",
    use_auth_token: Optional[str] = None
) -> pipeline:
    """
    Initialize the Falcon text-generation pipeline with quantization.
    No token required for public models.
    """
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        llm_int8_enable_fp32_cpu_offload=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        quantization_config=quantization_config,
        offload_folder=offload_folder,
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        padding_side="right",
        truncation=True,
        trust_remote_code=True
    )

    falcon_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        repetition_penalty=repetition_penalty,
        torch_dtype=torch.float16
    )

    falcon_pipe.model.config.pad_token_id = falcon_pipe.model.config.eos_token_id
    return falcon_pipe

In [ ]:
# def format_falcon_output(raw_text):
#     """
#     Formats the Falcon model's raw output into structured sections.
#     """
#     sections = raw_text.split("\n\n")  # Separate into paragraphs
#     main_answer = sections[0] if sections else "No answer generated."
#     key_points = sections[1:] if len(sections) > 1 else []

#     formatted_output = f"**Main Answer:**\n{main_answer.strip()}\n\n**Key Points:**\n"
#     for point in key_points:
#         formatted_output += f"- {point.strip()}\n"
#     return formatted_output


def format_falcon_output(raw_text):
    if not raw_text.strip():
        return "**Main Answer:**\nNo answer could be generated.\n\n**Key Points:**\n- No details available."

    sections = raw_text.split("\n\n")
    main_answer = sections[0].strip() if sections else "No answer generated."
    key_points = list(set(sections[1:]))  # Remove duplicates

    formatted_output = f"**Main Answer:**\n{main_answer}\n\n**Key Points:**\n"
    if key_points:
        for point in key_points:
            formatted_output += f"- {point.strip()}\n"
    else:
        formatted_output += "- No key points available.\n"

    return formatted_output



In [ ]:
def further_chunk_gdpr_content(company_name: str, initial_documents: List[Document]) -> List[Document]:
    """Further split GDPR content into smaller chunks while preserving metadata."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=50)
    gdpr_chunks = []

    for document in initial_documents:
        sub_chunks = text_splitter.split_text(document.page_content)
        for sub_chunk in sub_chunks:
            gdpr_chunks.append(Document(
                page_content=sub_chunk,
                metadata={
                    "header": document.metadata.get("header"),
                    "document_type": "GDPR"
                }
            ))
    print(f"[DEBUG] Further chunked GDPR content for company {company_name}")
    return gdpr_chunks

def prepare_documents_for_vectorstore(chunks: List[Document]) -> List[Document]:
    """Prepare documents for vector store with consistent metadata handling."""
    documents = []

    for idx, chunk in enumerate(chunks):
        if isinstance(chunk, dict):
            chunk = Document(page_content=chunk['page_content'], metadata=chunk['metadata'])

        doc_type = chunk.metadata.get("document_type", "Unknown")

        metadata_company_name = chunk.metadata.get("company_name", "").lower()
        uuid = chunk.metadata.get("uuid", "")

        metadata = {
            "chunk_number": idx + 1,
            "document_type": doc_type,
            "company_name": metadata_company_name,
            "header": chunk.metadata.get("header", ""),
            "doc_link": (
                "https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32016R0679"
                if doc_type == "GDPR"
                else f"/content/drive/MyDrive/210 Capstone/policy/{metadata_company_name}.pdf"
            )
        }

        documents.append(Document(
            page_content=chunk.page_content,
            metadata=metadata
        ))
        print(f"[DEBUG] Prepared document {idx + 1} with metadata: {metadata}")

    return documents

def setup_vectorstore(documents: List[Document], embeddings, qdrant_url: str, qdrant_api_key: str) -> Optional[Qdrant]:
    """Initialize and populate the vector store using a Qdrant cloud instance."""
    try:
        # Connect to the Qdrant client
        qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
        print("[DEBUG] Connected to Qdrant cloud instance")

        collection_name = "policy_pulse_db"

        test_embedding = embeddings.embed_query("")
        #print(f"[DEBUG] Embedding output type: {type(test_embedding)}")
        #print(f"[DEBUG] Embedding output sample: {test_embedding}")

        test_embedding = embeddings.embed_query("")
        if isinstance(test_embedding, list):
          test_embedding = np.array(test_embedding)
        embedding_size = test_embedding.shape[0]
        print(f"[DEBUG] Detected embedding size: {embedding_size}")

        # Recreate the collection
        try:
            qdrant_client.recreate_collection(
                collection_name=collection_name,
                vectors_config={
                    "size": embedding_size,
                    "distance": "Cosine"
                }
            )
            print(f"[DEBUG] Collection '{collection_name}' recreated successfully")
        except Exception as e:
            print(f"[ERROR] Failed to recreate collection: {e}")
            return None

        # Add documents to the vectorstore
        try:
            vectorstore = Qdrant(
                client=qdrant_client,
                collection_name=collection_name,
                embeddings=embeddings
            )
            vectorstore.add_documents(documents)
            print("[DEBUG] Documents added to Qdrant collection")
            return vectorstore
        except Exception as e:
            print(f"[ERROR] Failed to add documents to vectorstore: {e}")
            return None

    except Exception as e:
        print(f"[ERROR] Error setting up vector store: {e}")
        return None


def create_policy_prompt_template() -> ChatPromptTemplate:
    """
    Creates a ChatPromptTemplate for the GDPR compliance expert.

    Returns:
        ChatPromptTemplate: Configured template for generating structured answers.
    """
    prompt_role = "You are a GDPR compliance expert tasked with providing accurate, structured answers."
    prompt_task = "Using only the provided context, generate a comprehensive response that includes all required sections."
    prompt_context = "Context:\n{context}"
    prompt_question = "Question:\n{question}"
    prompt_instruction = """Answer the question by referring specifically to the context.
    Structure your response exactly as follows:

    Main Answer:
    [Provide a concise answer using the context information]

    Key Points:
    • [List key points as bullet points]
    • [Each point should start with a bullet point]
    • [Extract at least 3-4 key points]

    GDPR References:
    [Must include specific GDPR articles mentioned or most relevant to the context. Use format: 'Article X - Title']"""

    # Combine all parts into the template
    policy_pulse_template = f"""<s>[INST] {prompt_role}
    {prompt_task}

    {prompt_context}

    {prompt_question}

    {prompt_instruction} [/INST]"""

    # Return the ChatPromptTemplate object
    return ChatPromptTemplate.from_template(policy_pulse_template)

# def create_policy_prompt_template() -> ChatPromptTemplate:
#     """
#     Creates a ChatPromptTemplate for the GDPR compliance expert.
#     """
#     prompt_role = "You are a GDPR compliance expert who MUST answer in the exact format specified."
#     prompt_task = """Using ONLY the provided context, generate a structured response.
#     You must ALWAYS use the format specified below, even if you feel the context is insufficient."""

#     prompt_context = "Context:\n{context}"
#     prompt_question = "Question:\n{question}"
#     prompt_instruction = """You MUST structure your response EXACTLY as follows:

#     Main Answer:
#     [Write a concise 2-3 sentence answer using ONLY information from the context]

#     Key Points:
#     • [Start each point with a bullet]
#     • [Include at least 3 points from the context]
#     • [Be specific and cite actual details from context]

#     GDPR References:
#     [List any GDPR articles mentioned in context. If none mentioned, write "No specific GDPR articles referenced in provided context."]"""

#     policy_pulse_template = f"""<s>[INST] {prompt_role}
#     {prompt_task}

#     {prompt_context}

#     {prompt_question}

#     {prompt_instruction} [/INST]"""

#     return ChatPromptTemplate.from_template(policy_pulse_template)

def classify_question_type(company_name, question: str) -> tuple:
    """
    Classify if a question is about a specific company policy or general GDPR.
    Returns "Policy" if a company name is identified in the question,
    otherwise returns "GDPR".

    Args:
        question: The question text

    Returns:
        Tuple: ("Policy" or "GDPR", matched company name or None)
    """
   #company_name = "bitpay"
    if company_name in question:
      company_name = company_name.lower()
    return "Policy" if company_name else ("GDPR", None)

def clean_generated_response(response: str) -> str:
    """
    This function cleans the generated response to remove any unwanted content
    such as initial instructions or repeated context.
    """
    cleaned_response = re.sub(r'Human: <s>\[INST\].*?\[/INST\]', '', response, flags=re.DOTALL)

    cleaned_response = re.sub(r'Context:\s*{.*?}\n', '', cleaned_response, flags=re.DOTALL)

    cleaned_response = re.sub(r'Question:\s*{.*?}\n', '', cleaned_response, flags=re.DOTALL)

    cleaned_response = cleaned_response.strip()

    return cleaned_response


def process_single_question(company, question_type,  question, qdrant_vectorstore, gdpr_retriever, rag_chain_falcon):
    """
    Process a single question using the RAG system and return the generated answer.

    Args:
        question (str): The question to process.
        qdrant_vectorstore: Qdrant vector store for policy documents.
        gdpr_retriever: Retriever for GDPR-related documents.
        rag_chain_falcon: RAG chain pipeline for generating answers.

    Returns:
        dict: A result including the question, context, generated answer, and document sources.
    """
    print(f"[PROCESSING] Question: {question}")

    dynamic_company_name = company
    #question_type = classify_question_type(dynamic_company_name, question)
    print(f"extracted company name: {dynamic_company_name}")
    print(f"extracted question_type: {question_type}")

    gdpr_results = []
    policy_results = []

    if question_type == 'GDPR':
        print(f"[PROCESSING] Retrieving GDPR content for question: {question}")
        gdpr_results = gdpr_retriever.get_relevant_documents(
            question, metadata_filters={"document_type": question_type}
        )

    elif question_type == 'Policy':
        print(f"[PROCESSING] Retrieving Policy content for question: {question} ({dynamic_company_name})")
        combined_filter = {"company_name": dynamic_company_name}
        policy_retriever = qdrant_vectorstore.as_retriever(search_kwargs={'filter': combined_filter})
        # policy_results = policy_retriever.get_relevant_documents(question)
        policy_results = qdrant_vectorstore.as_retriever(search_kwargs={'filter': {'document_type': 'Policy', 'company_name': dynamic_company_name}}).get_relevant_documents(question)


        if not policy_results:
            # print(f"[WARNING] No policy documents found for '{dynamic_company_name}'. Using broader retrieval.")
            # policy_results = qdrant_vectorstore.as_retriever().get_relevant_documents(question)
            print("[WARNING] No policy documents found. Broadening retrieval scope.")
            policy_results = qdrant_vectorstore.as_retriever().get_relevant_documents(question)


        print(f"[INFO] Retrieving GDPR articles for the question using semantic similarity")
        gdpr_results = gdpr_retriever.get_relevant_documents(question)


        print("[DEBUG] Retrieved Policy Context Snippets:")
        if policy_results:
            for i, doc in enumerate(policy_results):
                print(f"Snippet {i+1}: {doc.page_content[:200]}...")
        else:
            print("[DEBUG] No policy context snippets retrieved.")

    else:
        print(f"[ERROR] Unable to classify question type.")
        return {
            "question": question,
            "contexts": "No relevant context found.",
            "answer": "Unable to classify question type.",
            "doc_source": "N/A"
        }

    context_snippets = [doc.page_content[:500] for doc in policy_results + gdpr_results if doc.page_content]
    context_snippets_str = " ".join(context_snippets)

    document_sources = [doc.metadata.get('doc_link', 'N/A') for doc in policy_results + gdpr_results]
    document_sources_str = ", ".join(document_sources)


    # Add debug prints after retrieving documents
    context_snippets = [doc.page_content[:500] for doc in policy_results + gdpr_results if doc.page_content]
    context_snippets_str = " ".join(context_snippets)

    print(f"\n[DEBUG] Retrieved Context Snippets:")
    for i, snippet in enumerate(context_snippets):
        print(f"\nSnippet {i+1}:")
        print(snippet[:200] + "...")


    if not context_snippets_str:
        print(f"[ERROR] No context retrieved for question: {question}.")
        return {
            "question": question,
            "contexts": "No relevant context found for answering the question.",
            "answer": "No answer generated.",
            "doc_source": "N/A"
        }

    # rag_input = {"context": context_snippets_str, "question": question}
    # rag_input = {
    # "context": context_snippets_str,
    # "question": question,
    # "instructions": (
    #     "As a GDPR compliance expert, provide a detailed answer in the following format:\n"
    #     "**Main Answer:**\n[One concise sentence summarizing the key information.]\n\n"
    #     "**Key Points:**\n- [Point 1]\n- [Point 2]\n\n"
    #     "Reference GDPR articles where applicable."
    # )
    # }

    # rag_input = {
    # "context": context_snippets_str,
    # "question": question,
    # "instructions": (
    #     "Provide a GDPR-compliant answer to the question using the given context. "
    #     "Structure the response as follows:\n"
    #     "**Main Answer:** [Provide a concise, factual summary.]\n"
    #     "**Key Points:**\n- [Key Point 1]\n- [Key Point 2]\n"
    #     "Avoid redundancy and ensure relevance."
    # )}


    rag_input = {
    "context": context_snippets_str,
    "question": question,
    "instructions": (
        "Using the provided context, answer the question with specific details. Focus on 'data storage and processing' locations, "
        "GDPR compliance measures, and any relevant guidelines."
    )}

    response = rag_chain_falcon.invoke(rag_input)
    generated_answer = clean_generated_response(response)

    # Apply the formatter here
    formatted_answer = format_falcon_output(generated_answer)

    return {
        "question": question,
        "contexts": context_snippets_str,
        "answer": generated_answer,
        "answer": formatted_answer,
        "doc_source": document_sources_str
    }


def main():
    """
    Main function to process a policy document, refine chunks, interact with vector store,
    and generate responses using the RAG pipeline.
    """


    input_data = {
        "company_name": "stripe",
        "uuid": "123e4567-e89b-12d3-a456-426614174000",
        "question_type": "Policy",
        #"question": "What safeguards does Stripe implement for international data transfers?"
    }

    policy_paths, gdpr_path, company_name, uuid, document_type = fn_local_with_paths(input_data)

    policy_contents, gdpr_contents = [], []

    if document_type == "Policy":
        policy_contents, _ = load_pdf_content_with_page_count(policy_paths)
        print(f"Total pages in Policy Document ({Path(policy_paths).name}): {len(policy_contents)}")
    elif document_type == "GDPR":
        gdpr_contents, _ = load_pdf_content_with_page_count(gdpr_path)
        print(f"Total pages in GDPR PDF: {len(gdpr_contents)}")
    else:
        raise ValueError(f"Unsupported document type: {document_type}")

    print(f"Policy Path: {policy_paths if document_type == 'Policy' else 'N/A'}")
    print(f"GDPR Path: {gdpr_path if document_type == 'GDPR' else 'N/A'}")
    print(f"Company Name: {company_name}")
    print(f"UUID: {uuid}")

    # Process the uploaded policy
    if document_type == "Policy":
      result = process_policy(policy_paths)
      if result["status"] == "success":
          print(f"\nProcessed Result for {result['pdf_path']}:")
          print(f"Total number of chunks created: {result['total_chunks']}\n")
          for chunk in result["chunks"]:
              print(f"Header: {chunk['header']}")
              print(f"Content Snippet: {chunk['content'][:150]}...\n")
      else:
          print(f"Error processing {result['pdf_path']}: {result['message']}")
          return
    else:
        print("[INFO] Document type is not 'Policy'; skipping policy processing.")

    refined_chunks = []
    if document_type == "Policy":
        result = process_policy(policy_paths)
        if result["status"] == "success":
            print(f"\nProcessed Result for {result['pdf_path']}:")
            print(f"Total number of chunks created: {result['total_chunks']}\n")
            for chunk in result["chunks"]:
                print(f"Header: {chunk['header']}")
                print(f"Content Snippet: {chunk['content'][:150]}...\n")
        else:
            print(f"Error processing {result['pdf_path']}: {result['message']}")
            return

        refined_chunks = further_chunk_policy(company_name, uuid, result)
        print(f"Total number of sub-chunks created: {len(refined_chunks)}\n")
        for i, final_chunk in enumerate(refined_chunks, 1):
            print(f"Chunk {i}/{len(refined_chunks)}")
            print(f"PDF Path: {final_chunk['metadata']['pdf_path']}")
            print(f"Header: {final_chunk['metadata']['header']}")
            print(f"Sub-chunk {final_chunk['metadata']['chunk_index']}/{final_chunk['metadata']['total_sub_chunks']}")
            print(f"Content Snippet: {final_chunk['page_content'][:150]}...\n")
    elif document_type == "GDPR":
        gdpr_sentence_chunks = chunk_gdpr_by_section(gdpr_contents)
        #print("\nSample chunks:")
        for doc in gdpr_sentence_chunks[:5]:
            print(f"\nHeader: {doc.metadata['header']}")
            print(f"Content Snippet: {doc.page_content[:150]}...")

    else:
        print("[ERROR] Invalid document type provided.")


    huggingface_token = userdata.get('hf')
    falcon_pipe = initialize_falcon_pipeline(
        model_name="tiiuae/falcon-7b-instruct",
        max_new_tokens=1000,
        temperature=0.55,
        top_p=0.9,
        repetition_penalty=1.3,
        offload_folder="offload_folder",
        use_auth_token=huggingface_token
        )
    falcon_llm_lc = HuggingFacePipeline(pipeline=falcon_pipe)

    #gdpr_final_chunks = further_chunk_gdpr_content(company_name, gdpr_sentence_chunks)
    if document_type == "GDPR":
        gdpr_final_chunks = further_chunk_gdpr_content(company_name, gdpr_sentence_chunks)
    else:
        gdpr_final_chunks = []

    all_chunks = gdpr_final_chunks + refined_chunks
    all_documents = prepare_documents_for_vectorstore(all_chunks)
    print("[DEBUG] Prepared Documents for Vector Store:")
    for idx, document in enumerate(all_documents):
        print(f"Document {idx + 1}:")
        print(f"Page Content: {document.page_content[:100]}...")
        print(f"Metadata: {document.metadata}")
        print("-" * 50)

    QDRANT_URL = "https://5ccc316a-ba45-4910-b5f9-15eb181ae895.us-east4-0.gcp.cloud.qdrant.io:6333"

    qdrant_vectorstore = setup_vectorstore(
        documents=all_documents,
        embeddings=base_embeddings,
        qdrant_url=QDRANT_URL,
        qdrant_api_key=QDRANT_API_KEY
    )

    combined_filter = {"document_type": "GDPR"}
    gdpr_retriever = qdrant_vectorstore.as_retriever(search_kwargs={'filter': combined_filter})
    print(f"[TEST] Applying retrieval filter with company_name: '{company_name}'")

    policy_prompt = create_policy_prompt_template()

    rag_chain_falcon = (
        {
            "context": RunnablePassthrough(),
            "question": RunnablePassthrough()
        }
        | policy_prompt
        | falcon_llm_lc
        | RunnablePassthrough()
    )


    user_question = input("Ask your question: ")
    result = process_single_question(
        company_name,
        document_type,
        question=user_question,
        qdrant_vectorstore=qdrant_vectorstore,
        gdpr_retriever=gdpr_retriever,
        rag_chain_falcon=rag_chain_falcon
    )

    print("\nReturning the following to the UI:")
    print()
    print(result['answer'])
    print()
    print(result['doc_source'])

    return {
        "answer": result["answer"],
        "doc_source": result["doc_source"]
    }



if __name__ == "__main__":
    response = main()

Received input data:  {'company_name': 'stripe', 'uuid': '123e4567-e89b-12d3-a456-426614174000', 'question_type': 'Policy'}
Policy file located at: /content/drive/MyDrive/210 Capstone/policy/stripe.pdf
GDPR document located at: /content/drive/MyDrive/210 Capstone/policy/GDPR.pdf
Total pages in Policy Document (stripe.pdf): 29
Policy Path: /content/drive/MyDrive/210 Capstone/policy/stripe.pdf
GDPR Path: N/A
Company Name: stripe
UUID: 123e4567-e89b-12d3-a456-426614174000
calculating potential headers

Processed Result for /content/drive/MyDrive/210 Capstone/policy/stripe.pdf:
Total number of chunks created: 88

Header: Last updated: January 16, 2024
Content Snippet: Last updated: January 16, 2024
This Privacy Policy includes important information about your personal data and we encourage you to read
it carefully....

Header: Welcome
Content Snippet: Welcome
We provide financial infrastructure for the internet. Individuals and businesses of all sizes use our
technology and services to fac

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


[DEBUG] Prepared document 1 with metadata: {'chunk_number': 1, 'document_type': 'Policy', 'company_name': 'stripe', 'header': 'Last updated: January 16, 2024', 'doc_link': '/content/drive/MyDrive/210 Capstone/policy/stripe.pdf'}
[DEBUG] Prepared document 2 with metadata: {'chunk_number': 2, 'document_type': 'Policy', 'company_name': 'stripe', 'header': 'Welcome', 'doc_link': '/content/drive/MyDrive/210 Capstone/policy/stripe.pdf'}
[DEBUG] Prepared document 3 with metadata: {'chunk_number': 3, 'document_type': 'Policy', 'company_name': 'stripe', 'header': 'Welcome', 'doc_link': '/content/drive/MyDrive/210 Capstone/policy/stripe.pdf'}
[DEBUG] Prepared document 4 with metadata: {'chunk_number': 4, 'document_type': 'Policy', 'company_name': 'stripe', 'header': 'Privacy Policy', 'doc_link': '/content/drive/MyDrive/210 Capstone/policy/stripe.pdf'}
[DEBUG] Prepared document 5 with metadata: {'chunk_number': 5, 'document_type': 'Policy', 'company_name': 'stripe', 'header': 'Privacy Policy', 'd

<ipython-input-40-1a61bc7b3a3c>:73: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


[DEBUG] Collection 'policy_pulse_db' recreated successfully
[DEBUG] Documents added to Qdrant collection
[TEST] Applying retrieval filter with company_name: 'stripe'
Ask your question: What personal data does Stripe collect from End Users of Link?
[PROCESSING] Question: What personal data does Stripe collect from End Users of Link?
extracted company name: stripe
extracted question_type: Policy
[PROCESSING] Retrieving Policy content for question: What personal data does Stripe collect from End Users of Link? (stripe)


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[INFO] Retrieving GDPR articles for the question using semantic similarity
[DEBUG] Retrieved Policy Context Snippets:
Snippet 1: a. Personal Data we collect about End Users
Using Link or Connecting your bank account.
Stripe offers a service called "Link," which allows you to
store your payment methods with Stripe to convenientl...
Snippet 2: FACTS
WHAT DOES STRIPE DO WITH YOUR PERSONAL INFORMATION?...
Snippet 3: Transactions
. When you use Link to make payments, we use your Personal Data (such as name, contact
information, payment method details) saved with us to complete transactions with Stripe Business Use...
Snippet 4: a. Personal Data we collect about End Customers
11/26/24, 8:51 PM
Privacy Policy
https://stripe.com/privacy
5/29
Transaction Data.
If you're an End Customer making payments to, receiving refunds from,...

[DEBUG] Retrieved Context Snippets:

Snippet 1:
a. Personal Data we collect about End Users
Using Link or Connecting your bank account.
Stripe offers a service call